In [1]:
import torch
import numpy as np
import torch.nn.functional as F
import random
from sklearn.datasets import load_iris
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from math import*

In [2]:
#此處 M,D指係數矩陣
def legendre(x,M):
    return torch.matmul(M,torch.Tensor([[1],[x],[0.5*(3*(x**2)-1)],[0.5*(5*(x**3)-3*x)]]))
def dlegendre(x,D):
    return torch.matmul(D,torch.Tensor([[0],[1],[3*x],[0.5*(15*(x**2)-3)]]))

In [3]:
class my_dataset(Dataset):
    def __init__(self):
        super().__init__()
        self.X,self.y = load_iris(return_X_y=True)

    def __len__(self):
        return len(self.y)
    
    def __getitem__(self,index):
        return torch.Tensor(self.X[index]),torch.LongTensor([self.y[index]])

bs = 10 #batch_size
all_data = my_dataset()
my_dataloader = DataLoader(all_data, batch_size = bs, num_workers = 0, drop_last = True, shuffle = True)

all_xdata = []
all_ydata = []

for x,y in my_dataloader:
    all_xdata.append(x)
    all_ydata.append(y)
    
X_test =  all_xdata[0:2]
y_test =  all_ydata[0:2]

X_train = all_xdata[2:]
y_train = all_ydata[2:]

In [4]:
class ODE_block(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.h1=torch.nn.Linear(4,4)
        self.h2=torch.nn.Linear(4,4)
        self.h3=torch.nn.Linear(4,4)
        self.h4=torch.nn.Linear(4,4)
    def forward(self,x):
        x1=F.relu(self.h1(x))
        x2=F.relu(self.h2(x1))
        x3=F.relu(self.h3(x2))
        x4=F.relu(self.h4(x3))
        self.x1=x1
        self.x2=x2
        self.x3=x3
        self.x4=x4
        return x4
up_layer = [torch.nn.Linear(4,4)]
middle_layer = [ODE_block()]
down_layer = [torch.nn.Linear(4,3),torch.nn.LogSoftmax()]
net = torch.nn.Sequential(*up_layer,*middle_layer,*down_layer)
loss_func = torch.nn.NLLLoss()
print(net)

Sequential(
  (0): Linear(in_features=4, out_features=4, bias=True)
  (1): ODE_block(
    (h1): Linear(in_features=4, out_features=4, bias=True)
    (h2): Linear(in_features=4, out_features=4, bias=True)
    (h3): Linear(in_features=4, out_features=4, bias=True)
    (h4): Linear(in_features=4, out_features=4, bias=True)
  )
  (2): Linear(in_features=4, out_features=3, bias=True)
  (3): LogSoftmax(dim=None)
)


In [5]:
#train ODE BLOCK
t = torch.Tensor([-1*(sqrt(525+70*sqrt(30)))/(35),-1*(sqrt(525-70*sqrt(30)))/(35),(sqrt(525-70*sqrt(30)))/(35),(sqrt(525+70*sqrt(30)))/(35)])
M = Variable(torch.Tensor(0.5*np.random.randn(bs,4,4)),requires_grad=True)
aopt = torch.optim.Adam(net[1].parameters(),lr=0.001)
aopt1 = torch.optim.Adam(net[0].parameters(),lr=0.001)
aopt2 = torch.optim.Adam(net[2].parameters(),lr=0.001)
gama = 0.5
lr = 0.001
epoch = 600

for itr in range(epoch):
    if itr % 10 < 5:
        pick = random.randrange(0,len(X_train))
        x = X_train[pick]
        pd = net(x)
        loss1 = gama*loss_func(pd,y_train[pick].squeeze())#gama為超參數
        loss2 = 0
        tp = [net[1].x1,net[1].x2,net[1].x3,net[1].x4]
        for j in range(len(t)):
            temp = dlegendre(t[j],M).squeeze()
            if j == 0 or j == 3:
                loss2 += ((18-sqrt(30))/36)*torch.mean(torch.norm(tp[j]-temp,dim=1))
            else:
                loss2 += ((18+sqrt(30))/36)*torch.mean(torch.norm(tp[j]-temp,dim=1))
        total_loss = loss1+loss2
        if total_loss < 1e-5:
            print("Finished...")
            break
        total_loss.backward(retain_graph = True)
        M.data.zero_()
        M.data.sub_(lr*M.grad.data)
    
        aopt.zero_grad()
        loss2.backward()
        aopt.step()
    else:
        pick = random.randrange(0,len(X_train))
        x = X_train[pick]
        pd = net(x)
        loss = loss_func(pd,y_train[pick].squeeze())
        aopt1.zero_grad()
        aopt2.zero_grad()
        loss.backward()
        aopt1.step()
        aopt2.step()
        
print("Training completed...")

c:\programdata\miniconda3\lib\site-packages\torch\nn\modules\container.py:119: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Training completed...


In [6]:
def class_predict(x):
    l = []
    for i in x:
        temp = net(i)
        for i in temp:
            i = list(i)
            l.append(i.index(max(i)))
    return l

def cal_accuracy(yp,yt):
    l = np.array([])
    for i in yt:
        l = np.append(l,np.array(i).squeeze())
    ans = list(np.array(yp)-l)
    print("accuracy:%.2f"%(ans.count(0)/len(ans)))

In [7]:
l=class_predict(X_test)
print(l)
cal_accuracy(l,y_test)

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
accuracy:0.30
